##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 多腕バンディットの基礎

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/intro_bandit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/agents/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/agents/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/agents/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

## 前書き

多腕バンディット（MAB）は、長期的な累積報酬を最大化するためにエージェントがアクション（アーム）を選択する必要のある機械学習フレームワークです。ラウンドごとに、エージェントは現在の状態（コンテキスト）に関する情報を受け取り、その情報と前のラウンドで収集したエクスペリエンスを基にアクションを選択します。各ラウンドの最後、エージェントは選択したアクションに関連する報酬を受け取ります。

おそらく最も純粋な例は、MAB の名に由来する問題でしょう。`k` 台のスロットマシン（ワンアームバンディット）があり、それほど大金を失うことなく、どの台のペイアウトが最も良いかを調べる必要があるとしましょう。

![多腕バンディット](https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Las_Vegas_slot_machines.jpg/320px-Las_Vegas_slot_machines.jpg)

各マシンを 1 回ずつ回して最も支払いの多かった台を選ぶのは良い方法とは言えません。エージェントが最初に運のよい結果を出しても、全般的には最適な結果を出さない台を選択しかねないためです。代わりに、あまり良い結果を出さない台を繰り返し選んで、それらの台に関する情報を収集することが勧められます。これが、多腕バンディットの主な挑戦です。エージェントは、以前の知識の利用と探索をうまく混ぜ合わせて、最適なアクションを見逃さないようにする必要があります。

MAB のさらに実用的な例には、学習者が決定を下すたびに得るサイド情報が伴います。「コンテキスト」または「観測」と呼ぶのがこのサイド情報です。


## 多腕バンディットのと強化学習

TF-Agents ライブラリに MAB スイートがあるのはなぜでしょうか？RL と MAB の関係は？多腕バンディットは、強化学習の特殊ケースとして考えることができます。[RL の基礎](https://www.tensorflow.org/agents/tutorials/0_intro_rl)では、次のように説明されています。

*時間ステップごとに、エージェントはそのポリシー $\pi(a_t|s_t)$ に基づいて環境に対するアクションを実行します。ポリシーの $s_t$ は環境からのその時点での観測で、環境から報酬 $r_{t+1}$ と次の観測 $s_{t+1}$ を受け取ります。ポリシーを改善して報酬（リターン）の合計を最大化することが目標です。*

一般的な RL のケースでは、次の観測 $s_{t+1}$ は前の状態 $s_t$ とポリシーが実行するアクション $a_t$ によって異なります。この最後の部分が、MAB と RL を分け隔てる部分です。MAB では、次の状態、つまり観測は、エージェントが選択するアクションに左右されません。

この類似性により、TF-Agents に存在するすべての概念を再利用することができます。

- **環境**は観測を出力し、報酬によってアクションに応答します。
- **ポリシー**は、観測に基づいてアクションを出力します。
- **エージェント**は、前の観測-アクション-報酬のタプルに基づいて、ポリシーを繰り返し更新します。


## キノコ環境

説明の目的により、「キノコ環境」というトイの例を使用します。キノコデータセット（[Schlimmer, 1981](https://archive.ics.uci.edu/ml/datasets/Mushroom)）は、食用キノコと毒キノコのラベル付きサンプルで構成されています。キノコのさまざまな部位の形状、色、サイズのほか、香りなどのさまざまな特徴が含まれます。

![キノコ](https://archive.ics.uci.edu/ml/assets/MLimages/Large73.jpg)

キノコデータセットは、あらゆる教師あり学習データセットと同様に、文脈的 MAB 問題に変換可能です。[Riquelme et al. (2018)](https://arxiv.org/pdf/1802.09127.pdf) も使用している手法を使用します。この会話の中で、エージェントはキノコの特徴を受け取り、それを食べるかどうかを決定します。食用キノコを食べると+5 の報酬が与えられ、毒キノコを食べると同等の確率で +5 または -35 の報酬が与えられます。キノコを食べない場合の報酬は、そのキノコの種類に関係なく 0 となります。以下の表に、報酬の割り当てをまとめています。

> ```
>
> ```

```
       | edible | poisonous
```

-----------|--------|---------- 食べた場合  |     +5 | -35 / +5 食べなかった場合 |      0 |        0

```

```

## LinUCB エージェント

文脈的バンディット環境でうまく機能するには、観察を踏まえて、各アクションの報酬関数を適切に推定する必要があります。一可能性として、線形関数で報酬関数を推定することができます。つまり、アクション $i$ ごとに、推定

$r_{t, i} \sim \langle v_t, \theta_i\rangle$

が現実に可能な限り近くなるパラメータ $\theta_i\in\mathbb R^d$ を求めるということです。ここで、$v_t\in\mathbb R^d$ は時間ステップ $t$ で受け取るコンテキストです。次に、エージェントがその推定に高い自信を持っている場合は、$\arg\max_{1, ..., K}\langle v_t, \theta_k\rangle$ を選択して、最も高い期待報酬をえることができます。

上記で説明したとおり、推定報酬が最も高いアームを選択するだけでは良い戦略にはなりません。線形推定エージェントで活用と探索を組み合わせるにはさまざまな方法がありますが、最も有名なものの1つに、線形上限信頼区間（LinUCB）アルゴリズムがあります（[Li et al. 2010](https://arxiv.org/abs/1003.0146) などを参照）。LinUCB には、主に 2 つのビルディングブロックがあります（詳細は省略）。

1. 線形最小二乗ですべてのアームのパラメータの推定値を維持します: $\hat\theta_i\sim X^+_i r_i$。ここで、$X_i$ と $r_i$ は、$i$ が選択された場合のラウンドの重なり合ったスタックと報酬で、$()^+$ は疑似逆行列です。
2. 上記の推定値の逆共分散 $X_i^ \ topX_i$ によって定義される *confidenceellipsoids* を維持します。

LinUCB の主な考え方は、「不確実性に直面した楽観主義」と共通です。エージェントは、それらの推定値の分散に対応する量だけ推定値をブーストすることにより、探索を組み込みます。ここで登場するのが信頼楕円です。アームごとに、楽観的な推定は $\hat r_i = \max_{\theta\in E_i}\langle v_t, \theta\rangle$ で、$E_i$ は $\hat\theta_i$ 周辺の楕円です。エージェントは最も最適に見えるアーム $\arg\max_i\hat r_i$ を選択します。

もちろん上記の説明は直感的なものであっても、LinUCB が行うことを表面的に要約したものにすぎません。実装は、[こちら](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/lin_ucb_agent.py)のコードベースをご覧ください。

## 今後の内容

Bandits ライブラリに関するより詳細なチュートリアルについては、[バンディットのチュートリアル](https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/bandits_tutorial.ipynb)をご覧ください。すぐにライブラリを確認し始める場合は、[こちら](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits)をご覧ください。さらに、トレーニングを始めたいという方は、[こちら](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2)のエンドツーエンドの例をご覧ください。上記に説明した、[LinUCB を使ったキノコ環境](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2/train_eval_mushroom.py)も掲載されています。 